# Creazione modello text analysis
Ho visto che il decision tree è il modello che mi offre migliori performance, soprattutto con TfIdf e tokenizzazione nltk/regex con stemming PorterStemmer().


In [1]:
from my_tokenizer import MyTokenizer
import pandas as pd
import util_strings as utils

È necessaria la normalizzazione delle feature, dato che nel testo abbiamo i nomi delle squadre ed i soprannomi, vado a sostituire tutte le parole relative alla squadra di casa con home team e stessa cosa per la away team.
Questo viene fatto tramite il metodo feature_normalization()

In [2]:
mt = MyTokenizer(pd.read_csv(utils.completed_dataset, index_col=0))
mt.feature_normalization()

In [3]:
mt.clean_text()
vectorizer = True #se True TfidfVectorizer, se False CountVectorizer
X_train, X_test, y_train, y_test = mt.set_bigram_and_get_sets(vectorizer) 
path_vec = utils.TfidfVectorizer if vectorizer else utils.CountVectorizer
mt.save_vectorizer(path_vec)

## Classificazione del modello
Migliori risultati con Decision Tree.
Viene salvato il modello

In [4]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import sklearn.metrics as mtr

In [ ]:
E = []

model = DecisionTreeClassifier()

accuracy = 0
min_val = 0.831

while(accuracy < min_val):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    analysis = pd.DataFrame()
            
    report = mtr.classification_report(y_test, y_pred, output_dict=True, zero_division=0)
    if report['accuracy'] > accuracy: 
        accuracy = report['accuracy']
        print(accuracy)

In [8]:
"""from mlxtend.evaluate import bias_variance_decomp

mse, bias, var = bias_variance_decomp(model, X_train, y_train, X_test, y_test, loss='mse', num_rounds=200, random_seed=1)
"""

ModuleNotFoundError: No module named 'mlxtend'

In [13]:
analysis = {
    'Model': 'Model', 'Accuracy': report['accuracy'],
    'Avg Precision (macro)': report['macro avg']['precision'],
    'Avg Recall (macro)': report['macro avg']['recall'],
    'Avg F1-score (macro)': report['macro avg']['f1-score'],
    'Avg Precision (weighted)': report['weighted avg']['precision'],
    'Avg Recall (weighted)': report['weighted avg']['recall'],
    'Avg F1-score (weighted)': report['weighted avg']['f1-score']
}
analysis

{'Model': 'Model',
 'Accuracy': 0.8312883435582822,
 'Avg Precision (macro)': 0.8211910048282247,
 'Avg Recall (macro)': 0.8059748512750217,
 'Avg F1-score (macro)': 0.8124245745729123,
 'Avg Precision (weighted)': 0.8312407219357778,
 'Avg Recall (weighted)': 0.8312883435582822,
 'Avg F1-score (weighted)': 0.8300314766901935}

In [14]:
import pickle
file_name = utils.classificatorTfIdf if vectorizer else utils.classificatorCV_LR
with open(file_name, 'wb') as file:
    pickle.dump(model, file)